In [ ]:
import pandas as pd
%pylab inline
import matplotlib.gridspec as gridspec


In [ ]:
read_in = pd.read_csv('APP_phot_all_exps.csv')

In [ ]:
try:
    del phot_df
except:
    pass
phot_df = read_in.copy().set_index(['ID', 'Filter', 'T_Start', 'Exp_Length', 'DrizzleType'])

In [ ]:
#zmag  = {'F336W':23.46,'F438W':24.98,'F555W': 25.81, 'F814W': 24.67, 'F656N': 19.92}#[hst_filter]-0.1

In [ ]:
#-2.5*np.log10(phot_df['FLUX']) + np.array([zmag[w] for w in phot_df.index.get_level_values(1).values])-0.1 +2.5*np.log10(phot_df['ITIME'])

In [ ]:
# We select only the magnitudes from the single drizzled frames
phot_df = phot_df[phot_df.index.get_level_values(4)=='SingleDrizzle']

In [ ]:
# First look for stars without CRs in there
phot_df = phot_df[phot_df.CERROR=='NoError']

In [ ]:
# Only select stars with V band magnitudes between 15 and 20
F555W_mags = phot_df['MAG']
selection  = (F555W_mags < 22) * (F555W_mags > 16)
phot_df    = phot_df[selection]

In [ ]:
# Take only values with no large shift
phot_df    = phot_df[phot_df.XSHIFT**2+phot_df.YSHIFT**2 < 3]

In [ ]:
# Normalize observation time 
phot_df.loc[:,'Time'] = pd.to_numeric(phot_df.index.get_level_values(2))
phot_df.loc[:,'Time'] = phot_df['Time'] - phot_df['Time'].min()

In [ ]:
groupby    = phot_df.groupby(['ID', 'Filter', 'Exp_Length', 'DrizzleType'])['MAG'].std()

In [ ]:
# Select only stars with limited std in magnitudes
groupby    = phot_df.groupby(['ID', 'Filter', 'Exp_Length', 'DrizzleType'])['MAG'].std()
groupby    = groupby.groupby('ID').median()<0.3#groupby < 1.5
good_stars = groupby.index[groupby]
phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(0)]]
#phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(1)]]
#phot_df    = phot_df.droplevel([2,4]).loc[good_stars]

In [ ]:
# Select only stars with limited mag error
phot_df   = phot_df[phot_df['MERR']<0.1]

In [ ]:
# Select some stars with enough remaining measurements
groupby    = phot_df.reset_index(level=1).groupby('ID')['Filter'].nunique()>=5
good_stars = groupby.index[groupby]
phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(0)]]

groupby    = (phot_df.groupby('ID')['MAG'].nunique()>72)
good_stars = groupby.index[groupby]
phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(0)]]

In [ ]:
# Select those with std(Ha)> std(broadband)
star_filter_std = phot_df.groupby(['ID', 'Filter'])['MAG'].std()
Ha_std   = star_filter_std[star_filter_std.index.get_level_values(1)=='F656N']
mean_std = star_filter_std.groupby('ID').mean()
high_Ha_var_stars = mean_std.index[Ha_std.droplevel(1) > 1* mean_std].values

In [ ]:
plt.figure(figsize=(18,16))
gs1 = gridspec.GridSpec(3,3)
gs1.update(wspace=0, hspace=0) # set the spacing between axes. 
Nstars_plot = 9
filterlist = phot_df.index.get_level_values(1).unique()
for nstar, starID in enumerate(np.random.choice(high_Ha_var_stars, Nstars_plot, replace=False)):
    for filter_iter, filter_ in enumerate(filterlist):
        plt.subplot(gs1[nstar])
        data = phot_df.loc[starID, filter_]
        data = data[['MAG', 'MERR', 'Time']].sort_values(by='Time')
        mags = data['MAG']
        mags = mags - np.max(mags) + filter_iter
        merr = data['MERR']
        time = data['Time']
        plt.plot(time,mags, linewidth=1.2, label=filter_)
        plt.scatter(time,mags, c='red', s=5, label='')
        plt.errorbar(time, mags, yerr=merr, c='black', ls='none', label='')
        plt.title('Star '+str(int(starID)), pad=-25, size=20)
        plt.xlim(-1,8.3)
        
        #plt.ylim(15.,18.5)
        if nstar not in [0,3,6]:
            plt.yticks([])
        else:
            plt.ylabel('Shifted + normalized magnitude', size=16)
            plt.yticks(size=14)
        if nstar not in [6,7,8]:
            plt.xticks([])
        else:
            plt.xticks(size=16)
            plt.xlabel(r'$\Delta \, T$ [days]',size=16)

plt.tight_layout()
plt.legend(loc='upper center', bbox_to_anchor=(-0.5, -0.15),
          fancybox=True, shadow=True, ncol=5, fontsize=16)
plt.suptitle(r'Stellar H$\alpha$ brightness variability in 30-dor', y=1.03,fontsize=25)
plt.show()